In [1]:
import os
os.chdir("../")
%pwd

'g:\\My Drive\\Documents\\Coding\\Text-Summarizer'

In [3]:
## Create Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig(object):
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [8]:
## Update Config Manager

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager(object):
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=float(params.save_steps),
            gradient_accumulation_steps=params.gradient_accumulation_steps,
        )
        return model_trainer_config

In [5]:
## Create Component

from transformers import (AutoModelForSeq2SeqLM, 
                          AutoTokenizer, 
                          DataCollatorForSeq2Seq,
                          TrainingArguments, 
                          Trainer)
from datasets import load_from_disk
import torch


class ModelTrainer(object):
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model_ckpt = self.config.model_ckpt
        tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # loading dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # initialize training args
        training_args = TrainingArguments(self.config.root_dir, num_train_epochs=self.config.num_train_epochs, 
                                          warmup_steps=self.config.warmup_steps, 
                                          per_device_train_batch_size=self.config.per_device_train_batch_size, 
                                          per_device_eval_batch_size=self.config.per_device_train_batch_size,
                                          weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
                                          evaluation_strategy=self.config.evaluation_strategy, 
                                          eval_steps=self.config.eval_steps, save_steps=self.config.save_steps,
                                          gradient_accumulation_steps=self.config.gradient_accumulation_steps
                                          )
        # begin training
        trainer = Trainer(
            model=model_pegasus, args=training_args, 
            tokenizer=tokenizer, data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        # save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        # save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


c:\Users\ADMIN\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
## Pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-06-21 14:30:56,826: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-21 14:30:56,840: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-21 14:30:56,846: INFO: common: created directory at artifacts]
[2023-06-21 14:30:56,850: INFO: common: created directory at artifacts/model_trainer]


c:\Users\ADMIN\anaconda3\envs\textS\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  2%|▏         | 1/51 [04:13<3:31:37, 253.95s/it]

KeyboardInterrupt: 